# `Project Dependencies`

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00


# `Project Code`

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset
import time

# byte_pair_tokenization = ["openai-gpt", "gpt2", "NousResearch/Llama-2-13b-hf"]
# unigram_tokenization = ["google/bigbird-roberta-base", "facebook/mbart-large-50-many-to-many-mmt" , "albert-base-v2" , "xlnet-base-cased"]
# wordpiece_tokenization = ['distilbert-base-uncased','google/mobilebert-uncased','funnel-transformer/small-base','sentence-transformers/all-mpnet-base-v2']
# sentencepiece_tokenization = ["google/flan-t5-base"]

#corpus = ["I have a new GPU!", "I wonder how fast the model will train on this.","Hello there"]


dataset = load_dataset("HuggingFaceH4/self-instruct-seed")
corpus = dataset['train']['instruction']
#print(corpus)

def initialize_model(model_name:str):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  return tokenizer

def time_model(tokenizer_object, text):
    # Capture the start time
    start_time = time.time()
    tokenizer_object.encode_plus(text)
    # Capture the end time
    end_time = time.time()
    # Calculate the elapsed time in milliseconds
    elapsed_time = (end_time - start_time) * 1e3
    return elapsed_time

def analyse_encode_plus(tokenizers: list, corpus: list) -> dict:
    results = {}
    results['method'] = "Unbatched"

    for hugging_face_tokenizer in tokenizers:
        try:
            tokenizer = initialize_model(hugging_face_tokenizer)
            total_time = 0  # Total time for all lines
            for text in corpus:
                # Tokenize each line in the corpus individually
                line_time = time_model(tokenizer, text)
                total_time += line_time

            # Extract tokenizer name or path for dictionary key
            tokenizer_name = tokenizer.name_or_path
            results[tokenizer_name] = total_time
            print(f"{tokenizer_name}: {total_time} ms")  # Print the result
        except Exception as e:
            print(f"Error occurred for {hugging_face_tokenizer}: {e}")
            continue

    return results

def time_model_batch(tokenizer, corpus):
    """Calculate timing for batch_encode_plus"""
    # Capture the start time
    start_time = time.time()
    tokenizer.batch_encode_plus(corpus)
    # Capture the end time
    end_time = time.time()
    # Calculate the elapsed time in milliseconds
    elapsed_time = (end_time - start_time) * 1e3
    return elapsed_time


def analyse_batch(tokenizers: list, corpus: list) -> dict:
    results = {}
    results['method'] = "Batched"
    for hugging_face_tokenizer in tokenizers:
        tokenizer = initialize_model(hugging_face_tokenizer)
        try:
          average_time = time_model_batch(tokenizer, corpus)

          # Extract tokenizer name or path for dictionary key
          tokenizer_name = tokenizer.name_or_path
          results[tokenizer_name] = average_time
          print(f"{tokenizer_name}: {average_time} ms")  # Print the result
        except Exception as e:
          print(f"Error occured for {hugging_face_tokenizer}: {e}")
          continue

    return results


Updated Project Code With detokenisation

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset
import time

# byte_pair_tokenization = ["openai-gpt", "gpt2", "NousResearch/Llama-2-13b-hf"]
# unigram_tokenization = ["google/bigbird-roberta-base", "facebook/mbart-large-50-many-to-many-mmt" , "albert-base-v2" , "xlnet-base-cased"]
# wordpiece_tokenization = ['distilbert-base-uncased','google/mobilebert-uncased','funnel-transformer/small-base','sentence-transformers/all-mpnet-base-v2']
# sentencepiece_tokenization = ["google/flan-t5-base"]

#corpus = ["I have a new GPU!", "I wonder how fast the model will train on this.","Hello there"]

dataset = load_dataset("HuggingFaceH4/self-instruct-seed")
corpus = dataset['train']['instruction']

def initialize_model(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return tokenizer

#Unbatched data
def time_model(tokenizer_object, text, tokenize=True):
    if tokenize:
        # Capture the start time
        start_time = time.time()
        tokenizer_object.encode_plus(text)
    else:
        tokens = tokenizer_object.encode_plus(text)
        # Capture the start time
        start_time = time.time()
        tokenizer_object.decode(tokens['input_ids'], skip_special_tokens=True)

    # Capture the end time
    end_time = time.time()
    # Calculate the elapsed time in milliseconds
    elapsed_time = (end_time - start_time) * 1e3
    return elapsed_time

def analyse_encode_plus(tokenizers: list, corpus: list) -> dict:
    results = {}
    results['method'] = "Unbatched"

    for hugging_face_tokenizer in tokenizers:
        try:
            tokenizer = initialize_model(hugging_face_tokenizer)
            total_time_tokenization = 0  # Total time for tokenization
            total_time_detokenization = 0  # Total time for detokenization

            for text in corpus:
                # Tokenize each line in the corpus individually
                tokenization_time = time_model(tokenizer, text, tokenize=True)
                total_time_tokenization += tokenization_time

                # Detokenize the encoded text
                detokenization_time = time_model(tokenizer, text, tokenize=False)
                total_time_detokenization += detokenization_time

            # Extract tokenizer name or path for dictionary key
            tokenizer_name = tokenizer.name_or_path
            results[tokenizer_name] = {
                "Tokenization": total_time_tokenization,
                "Detokenization": total_time_detokenization
            }

            print(f"{tokenizer_name}:")
            print(f"Tokenization Time: {total_time_tokenization} ms")
            print(f"Detokenization Time: {total_time_detokenization} ms")
        except Exception as e:
            print(f"Error occurred for {hugging_face_tokenizer}: {e}")
            continue

    return results

def time_model_batch(tokenizer, corpus,tokenize=True):

    if tokenize:
        # Capture the start time
        start_time = time.time()
        tokenizer.batch_encode_plus(corpus)
    else:
        tokens = tokenizer.batch_encode_plus(corpus)
        # Capture the start time
        start_time = time.time()
        tokenizer.batch_decode(tokens['input_ids'], skip_special_tokens=True)

    # Capture the end time
    end_time = time.time()
    # Calculate the elapsed time in milliseconds
    elapsed_time = (end_time - start_time) * 1e3
    return elapsed_time

def analyse_batch(tokenizers: list, corpus: list) -> dict:
    results = {}
    results['method'] = "Batched"
    for hugging_face_tokenizer in tokenizers:
        tokenizer = initialize_model(hugging_face_tokenizer)
        try:
            #get time for tokenization
            tokenization_time = time_model_batch(tokenizer, corpus, tokenize=True)
            #get time for detokenization
            detokenization_time = time_model_batch(tokenizer, corpus,tokenize=False)

            # Extract tokenizer name or path for dictionary key
            tokenizer_name = tokenizer.name_or_path
            results[tokenizer_name] = {
                "Tokenization": tokenization_time,
                "Detokenization": detokenization_time
            }

            print(f"{tokenizer_name}:")
            print(f"Tokenization Time: {tokenization_time} ms")
            print(f"Detokenization Time: {detokenization_time} ms")
        except Exception as e:
          print(f"Error occured for {hugging_face_tokenizer}: {e}")
          continue

    return results


In [ ]:
byte_pair = ["openai-gpt", "gpt2", "NousResearch/Llama-2-13b-hf"]
print("--------------------------Unbatched Data-------------------------------")
byte_pair_timing_unbatched = analyse_encode_plus(byte_pair, corpus)
print("--------------------------Batched Data-------------------------------")
byte_pair_timing_batched = analyse_batch(byte_pair,corpus)

--------------------------Unbatched Data-------------------------------
openai-gpt:
Tokenization Time: 24.712324142456055 ms
Detokenization Time: 24.597883224487305 ms
gpt2:
Tokenization Time: 28.779983520507812 ms
Detokenization Time: 25.022029876708984 ms
NousResearch/Llama-2-13b-hf:
Tokenization Time: 22.885560989379883 ms
Detokenization Time: 30.135154724121094 ms
--------------------------Batched Data-------------------------------
openai-gpt:
Tokenization Time: 14.763593673706055 ms
Detokenization Time: 28.710603713989258 ms
gpt2:
Tokenization Time: 11.205911636352539 ms
Detokenization Time: 26.978492736816406 ms
NousResearch/Llama-2-13b-hf:
Tokenization Time: 10.309696197509766 ms
Detokenization Time: 25.06566047668457 ms


In [ ]:
# Get response time for word_piece models
word_piece =['distilbert-base-uncased','google/mobilebert-uncased','funnel-transformer/small-base','sentence-transformers/all-mpnet-base-v2']
print("--------------------------Unbatched Data-------------------------------")
word_piece_timing_unbatched = analyse_encode_plus(word_piece, corpus)
print("--------------------------Batched Data-------------------------------")
word_piece_timing_batched = analyse_batch(word_piece,corpus)

--------------------------Unbatched Data-------------------------------
distilbert-base-uncased:
Tokenization Time: 24.35016632080078 ms
Detokenization Time: 31.602859497070312 ms
google/mobilebert-uncased:
Tokenization Time: 28.52153778076172 ms
Detokenization Time: 36.28993034362793 ms
funnel-transformer/small-base:
Tokenization Time: 25.632143020629883 ms
Detokenization Time: 35.263776779174805 ms
sentence-transformers/all-mpnet-base-v2:
Tokenization Time: 25.9096622467041 ms
Detokenization Time: 34.56592559814453 ms
--------------------------Batched Data-------------------------------
distilbert-base-uncased:
Tokenization Time: 13.002872467041016 ms
Detokenization Time: 33.32972526550293 ms
google/mobilebert-uncased:
Tokenization Time: 12.912988662719727 ms
Detokenization Time: 29.02817726135254 ms
funnel-transformer/small-base:
Tokenization Time: 13.851165771484375 ms
Detokenization Time: 26.905536651611328 ms
sentence-transformers/all-mpnet-base-v2:
Tokenization Time: 11.75808906

In [ ]:
sentence_piece = ["google/flan-t5-base"]
sentence_piece_timing_unbatched = analyse_encode_plus(sentence_piece, corpus)
sentence_piece_timing_batched = analyse_batch(sentence_piece,corpus)

google/flan-t5-base:
Tokenization Time: 29.726028442382812 ms
Detokenization Time: 30.49778938293457 ms
google/flan-t5-base:
Tokenization Time: 14.184236526489258 ms
Detokenization Time: 25.858163833618164 ms


In [ ]:
unigram = ["google/bigbird-roberta-base", "facebook/mbart-large-50-many-to-many-mmt" , "albert-base-v2" , "xlnet-base-cased"]
print("--------------------------Unbatched Data-------------------------------")
unigram_timing_unbatched = analyse_encode_plus(unigram, corpus)
print("--------------------------Batched Data-------------------------------")
unigram_timing_batched = analyse_batch(unigram,corpus)

google/bigbird-roberta-base: 44.766902923583984 ms


facebook/mbart-large-50-many-to-many-mmt: 28.438806533813477 ms


albert-base-v2: 26.40247344970703 ms


xlnet-base-cased: 24.908065795898438 ms
google/bigbird-roberta-base: 0.12436458042689733 ms
facebook/mbart-large-50-many-to-many-mmt: 0.2441052028111049 ms
albert-base-v2: 0.2801050458635603 ms
xlnet-base-cased: 0.25938442775181364 ms


In [ ]:
print(byte_pair_timing_unbatched)
print(byte_pair_timing_batched)
print(word_piece_timing_unbatched)
print(word_piece_timing_batched)
print(sentence_piece_timing_unbatched)
print(sentence_piece_timing_batched)
print(unigram_timing_unbatched)
print(unigram_timing_batched)

{'method': 'Unbatched', 'openai-gpt': {'Tokenization': 24.712324142456055, 'Detokenization': 24.597883224487305}, 'gpt2': {'Tokenization': 28.779983520507812, 'Detokenization': 25.022029876708984}, 'NousResearch/Llama-2-13b-hf': {'Tokenization': 22.885560989379883, 'Detokenization': 30.135154724121094}}
{'method': 'Batched', 'openai-gpt': {'Tokenization': 14.763593673706055, 'Detokenization': 28.710603713989258}, 'gpt2': {'Tokenization': 11.205911636352539, 'Detokenization': 26.978492736816406}, 'NousResearch/Llama-2-13b-hf': {'Tokenization': 10.309696197509766, 'Detokenization': 25.06566047668457}}
{'method': 'Unbatched', 'distilbert-base-uncased': {'Tokenization': 24.35016632080078, 'Detokenization': 31.602859497070312}, 'google/mobilebert-uncased': {'Tokenization': 28.52153778076172, 'Detokenization': 36.28993034362793}, 'funnel-transformer/small-base': {'Tokenization': 25.632143020629883, 'Detokenization': 35.263776779174805}, 'sentence-transformers/all-mpnet-base-v2': {'Tokenizati